In [1]:
# things we need for NLP
import nltk
#nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

c:\users\yoran\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

182 documents
30 classes ['bacon', 'baconAlone', 'bigmac', 'bigmacAlone', 'colaLarge', 'colaMedium', 'colaMenu', 'colaSmall', 'done', 'fantaLarge', 'fantaMedium', 'fantaMenu', 'fantaSmall', 'goodbye', 'greeting', 'hambAlone', 'hamburger', 'inComplete', 'inCompleteDrink', 'menuLargeBacon', 'menuLargeBigMac', 'menuLargeHamb', 'menuMediumBacon', 'menuMediumBigMac', 'menuMediumHamb', 'spriteLarge', 'spriteMedium', 'spriteMenu', 'spriteSmall', 'thanks']
64 unique stemmed words ["'s", 'a', 'ad', 'al', 'an', 'anoth', 'anyon', 'ar', 'axtr', 'bacon', 'big', 'burg', 'bye', 'can', 'chamburg', 'col', 'crispy', 'crsipy', 'day', 'do', 'extr', 'fant', 'get', 'good', 'goodby', 'hamburg', 'hav', 'hello', 'help', 'hi', 'how', 'i', 'instead', 'is', 'it', 'just', 'larg', 'lat', 'lik', 'mac', 'med', 'menu', 'much', 'nee', 'no', 'okay', 'on', 'pleas', 'pric', 'sanwich', 'see', 'siz', 'smal', 'sprite', 'thank', 'that', 'the', 'ther', 'want', 'what', 'with', 'would', 'ye', 'you']


In [4]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
#reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 22999  | total loss: 0.36212 | time: 0.077s
| Adam | epoch: 1000 | loss: 0.36212 - acc: 0.7904 -- iter: 176/182
Training Step: 23000  | total loss: 0.36508 | time: 0.081s
| Adam | epoch: 1000 | loss: 0.36508 - acc: 0.7863 -- iter: 182/182
--
INFO:tensorflow:C:\Users\yoran\OneDrive - Hogeschool West-Vlaanderen\school\semester 4\machine learning\labo\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )


In [7]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [8]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\yoran\OneDrive - Hogeschool West-Vlaanderen\school\semester 4\machine learning\labo\chatbot\model.tflearn


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [10]:
p = bow("I want a large big mac menu",words)

In [11]:
print(p)

[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]


In [12]:
# create a data structure to hold user context
context = {'123':'none'}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    res = None
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                                      

                    # check if this intent is contextual and applies to this user's conversation)

                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        res = (random.choice(i['responses']))
                    # set context for this intent if necessary
                   
                    if 'context_set' in i:
                        print(i['context_set'])
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']
                    if res != None:
                        return print(res)

            results.pop(0)

In [13]:
print(context)

{'123': 'none'}


In [14]:
context = {'123':'none'}

In [15]:
classify("i want a big mac")

[('bigmac', 0.9895258)]

In [16]:
response("i want a big mac")

bigmacMenuQues
Would you like it in a menu


In [17]:
print(context)

{'123': 'bigmacMenuQues'}


In [18]:
response('yes a large one')

colaQues
colaQues
colaQues
colaQues
colaQues
colaQues


In [19]:
print(context)

{'123': 'colaQues'}


In [20]:
classify('no i want a fanta')

[('fantaMenu', 0.9768147)]

In [21]:
response('yes')

bigmacMenuQues
baconMenuQues
hambMenuQues


In [22]:
print(context)

{'123': 'hambMenuQues'}


In [23]:
response("how much is it")